In [2]:
import cv2
import numpy as np

video_path = "low_res_clip.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise ValueError("Error opening video file.")

ball_radius_range = (3, 7)
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=30, detectShadows=True)

kalman = cv2.KalmanFilter(4, 2)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.1

lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

prev_gray = None
prev_points = None

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('volleyball_tracking.avi', fourcc, 30.0, 
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    fg_mask = bg_subtractor.apply(gray)
    cv2.imshow("bg_subtractor", fg_mask)
    fg_mask = cv2.medianBlur(fg_mask, 5)
    cv2.imshow("median_blur", fg_mask)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    cv2.imshow("fg_mask_closed", fg_mask)
    cv2.imshow("fg_mask_threshold", fg_mask)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 50:
            cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)

    detected_ball = None
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if 50 < area < 500:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if ball_radius_range[0] < radius < ball_radius_range[1]:
                detected_ball = (int(x), int(y))
                break

    if detected_ball:
        x, y = detected_ball
        measurement = np.array([[np.float32(x)], [np.float32(y)]])
        kalman.correct(measurement)
        prediction = kalman.predict()
        predicted_pos = (int(prediction[0]), int(prediction[1]))

        cv2.circle(frame, (x, y), ball_radius_range[1], (0, 255, 0), 2)
        cv2.circle(frame, predicted_pos, ball_radius_range[1], (0, 0, 255), 2)

    circles = cv2.HoughCircles(fg_mask, cv2.HOUGH_GRADIENT, dp=1.2, minDist=10,
                               param1=10, param2=20, minRadius=ball_radius_range[0], maxRadius=ball_radius_range[1])
    
    if circles is not None:
        circles = np.uint16(np.around(circles))[0, :]
        ball_center = (circles[0][0], circles[0][1])

        measurement = np.array([[np.float32(ball_center[0])], [np.float32(ball_center[1])]])
        kalman.correct(measurement)
        prediction = kalman.predict()
        predicted_pos = (int(prediction[0]), int(prediction[1]))

        cv2.circle(frame, ball_center, ball_radius_range[1], (0, 255, 0), 2)
        cv2.circle(frame, predicted_pos, ball_radius_range[1], (0, 0, 255), 2)

        prev_points = np.array([[ball_center]], dtype=np.float32)

    if prev_gray is not None and prev_points is not None:
        next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)
        if status[0][0] == 1:
            ball_center = (int(next_points[0][0][0]), int(next_points[0][0][1]))
            cv2.circle(frame, ball_center, ball_radius_range[1], (255, 0, 0), 2)
            prev_points = next_points

    prev_gray = gray.copy()

    out.write(frame)

    cv2.imshow("Ball Tracking", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


C:\Users\harjo\AppData\Local\Temp\ipykernel_34356\3571030175.py:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_pos = (int(prediction[0]), int(prediction[1]))
C:\Users\harjo\AppData\Local\Temp\ipykernel_34356\3571030175.py:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_pos = (int(prediction[0]), int(prediction[1]))
